In [1]:
# Arnaud Guzman-Annès

# TSP - 9 locations

In [2]:
# Import libraries

import pulp as plp
from pulp import *
import numpy as np
from timeit import default_timer as timer


Using license file /Users/arnaudguzman-annes/gurobi.lic
Academic license - for non-commercial use only
No parameters matching '_test' found


In [3]:
# Costs

cost = np.array([[0, 45, 64, 25, 60, 32, 30, 70, 40],
                 [45, 0, 30, 42, 48, 25, 38, 50, 26],
                 [64, 30, 0, 20, 50, 37, 42, 50, 60],
                 [25, 42, 20, 0, 24, 56, 62, 27, 53],
                 [60, 48, 50, 24, 0, 28, 38, 46, 52],
                 [32, 25, 37, 56, 28, 0, 60, 49, 24],
                 [30, 38, 42, 62, 38, 60, 0, 64, 49],
                 [70, 50, 50, 27, 46, 49, 64, 0, 50],
                 [40, 26, 60, 53, 52, 24, 49, 50, 0]])


In [5]:
# Routes

I = range(0, cost.shape[0])
J = range(0, cost.shape[1])

x = plp.LpVariable.dicts(name='X', indexs=(I,J), cat=plp.LpBinary)


In [5]:
# Objective

prob = plp.LpProblem("Travel_Salesman_Problem")
prob.sense = plp.LpMinimize
prob += plp.lpSum(cost[i][j] * x[i][j] for i in I for j in J)


In [6]:
# Constraints - Part 1 - Enter and exit once

# (1) Salesman must enter each location exactly once:

for j in J:
    prob += plp.lpSum(x[i][j] for i in I) == 1
    
# (2) Salesman must exit each location exactly once:

for i in I:
    prob += plp.lpSum(x[i][j] for j in J) == 1


In [7]:
# Constraints - Part 2 - sub-tours

Y = range(0, cost.shape[0]-1)
y = plp.LpVariable.dicts(name='y', indexs=(Y), lowBound=0, upBound=cost.shape[0]-1, cat=plp.LpInteger)

# (1) Avoiding coming back to the same knot:

for i in I:
    prob += (x[i][i] == 0)

# (2) Avoiding sub-tours:

for i in Y:
    for j in Y:
        prob += y[i] - y[j] + cost.shape[0]*x[i][j] <= cost.shape[0]-1

In [8]:
prob

Travel_Salesman_Problem:
MINIMIZE
45*X_0_1 + 64*X_0_2 + 25*X_0_3 + 60*X_0_4 + 32*X_0_5 + 30*X_0_6 + 70*X_0_7 + 40*X_0_8 + 45*X_1_0 + 30*X_1_2 + 42*X_1_3 + 48*X_1_4 + 25*X_1_5 + 38*X_1_6 + 50*X_1_7 + 26*X_1_8 + 64*X_2_0 + 30*X_2_1 + 20*X_2_3 + 50*X_2_4 + 37*X_2_5 + 42*X_2_6 + 50*X_2_7 + 60*X_2_8 + 25*X_3_0 + 42*X_3_1 + 20*X_3_2 + 24*X_3_4 + 56*X_3_5 + 62*X_3_6 + 27*X_3_7 + 53*X_3_8 + 60*X_4_0 + 48*X_4_1 + 50*X_4_2 + 24*X_4_3 + 28*X_4_5 + 38*X_4_6 + 46*X_4_7 + 52*X_4_8 + 32*X_5_0 + 25*X_5_1 + 37*X_5_2 + 56*X_5_3 + 28*X_5_4 + 60*X_5_6 + 49*X_5_7 + 24*X_5_8 + 30*X_6_0 + 38*X_6_1 + 42*X_6_2 + 62*X_6_3 + 38*X_6_4 + 60*X_6_5 + 64*X_6_7 + 49*X_6_8 + 70*X_7_0 + 50*X_7_1 + 50*X_7_2 + 27*X_7_3 + 46*X_7_4 + 49*X_7_5 + 64*X_7_6 + 50*X_7_8 + 40*X_8_0 + 26*X_8_1 + 60*X_8_2 + 53*X_8_3 + 52*X_8_4 + 24*X_8_5 + 49*X_8_6 + 50*X_8_7 + 0
SUBJECT TO
_C1: X_0_0 + X_1_0 + X_2_0 + X_3_0 + X_4_0 + X_5_0 + X_6_0 + X_7_0 + X_8_0 = 1

_C2: X_0_1 + X_1_1 + X_2_1 + X_3_1 + X_4_1 + X_5_1 + X_6_1 + X_7_1 + X_8_1 = 1

_

In [97]:
# Solve and Time

start_time = timer()
prob.solve()
end_time = timer()

In [99]:
# The status of the solution is printed to the screen
print("="*30,"\nSolution Status:", plp.LpStatus[prob.status])

# Results
obj = value(prob.objective)
print("Optimal Result: ${}".format(obj))

print("TSP path:")
for v in prob.variables():
    if v.varValue != 0:
        if v.varValue == 1 and v.name != "y_0":
            print(v.name)

# Time
print("Time required:", round(end_time-start_time,4), "s")

Solution Status: Optimal
Optimal Result: $273.0
TSP path:
X_0_6
X_1_8
X_2_1
X_3_2
X_4_7
X_5_0
X_6_4
X_7_3
X_8_5
Time required: 0.4582 s
